

# HWK5: Summarization 

## Aditya Sahu  016237819

### 1. Explain the difference between ROUGE and BLEU in context of summarization


ROUGE and BLEU both are popular metrics used for evaluating the quality of text summarization.

While both metrics are used to evaluate the quality of summarization, ROUGE focuses on recall while BLEU focuses on precision.

ROUGE - Recall-Oriented Understudy for Gisting Evaluation

BLEU - Bilingual Evaluation Understudy 

**Bleu measures precision:** how much the words (and/or n-grams) in the machine generated summaries appeared in the human reference summaries.

**Rouge measures recall:** how much the words (and/or n-grams) in the human reference summaries appeared in the machine generated summaries.

ROUGE measures the overlap between the generated summary and the reference summary in terms of recall, while BLEU measures similarity in terms of n-gram overlap.

ROUGE is more recall-oriented, while BLEU is more focused on precision.

### 2. Perform Abstractive and Extractive summarization for any **Dialog Dataset** (such as this dataset - https://huggingface.co/datasets/knkarthick/dialogsum )


1.   You can fine-tune t5-small for the abstractive summarization. 
2.   You just need to perform extractive summarization on 1-2 examples. 





### Imports

In [1]:
!pip install transformers   
!pip install datasets
!pip install rouge_score
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 18.0 MB/s eta 0:00:

## Load dialogsum dataset

In [2]:
from datasets import load_dataset
train_dataset = load_dataset("knkarthick/dialogsum", '3.0.0', split='train[:1000]')  #truncating for memory limitation
test_dataset = load_dataset("knkarthick/dialogsum", '3.0.0', split='test[:1000]')  #truncating for memory limitation

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-c8fac5d84cd35861/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


There are two fields that you'll want to use:

- `article`: the text of the article which'll be the input to the model.
- `highlights`: a condensed version of `text` which'll be the model target.

Then take a look at an example:

In [3]:
train_dataset[5]

{'id': 'train_5',
 'dialogue': "#Person1#: Happy birthday, Aims!\n#Person2#: Thank you, Lisa.\n#Person1#: Here is a present for you. I hope you like it.\n#Person2#: Oh, great! I love it! You know I've been expecting this for a long time.\n#Person1#: I'm very glad to hear that.\n#Person2#: Come here ; let me introduce some friends to you.",
 'summary': 'Lisa gives Aims a birthday present and Aims loves it.',
 'topic': 'birthday'}

In [4]:
print("DIALOGUE")
print(train_dataset[5]['dialogue'])
print("SUMMARY")
print(train_dataset[5]['summary'])

DIALOGUE
#Person1#: Happy birthday, Aims!
#Person2#: Thank you, Lisa.
#Person1#: Here is a present for you. I hope you like it.
#Person2#: Oh, great! I love it! You know I've been expecting this for a long time.
#Person1#: I'm very glad to hear that.
#Person2#: Come here ; let me introduce some friends to you.
SUMMARY
Lisa gives Aims a birthday present and Aims loves it.


## Abstractive Summarization

We will fine-tune a T5-small model for our abstractive summarization

### Preprocess

In [5]:
from transformers import AutoTokenizer
checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [6]:
prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    labels = tokenizer(text_target=examples['summary'], max_length=64, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [7]:
tokenized_dataset= train_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [8]:
tokenized_test_dataset= test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Now, the same article and its highlights we saw in the previous output, is tokenized. 

In [9]:
print("-"*40 + " ARTICLE " + "-"*40)
print(tokenized_dataset[1]['dialogue'])
print("-"*40 + " HIGHLIGHTS " + "-"*40)
print("\n"+tokenized_dataset[1]['summary'])
print("-"*40 + " ARTICLE TOKENS " + "-"*40)
print("\n"+ str(tokenized_dataset[1]['input_ids']))
print("-"*40 + " HIGHLIGHTS TOKENS " + "-"*40)
print("\n"+ str(tokenized_dataset[1]['labels']))

---------------------------------------- ARTICLE ----------------------------------------
#Person1#: Hello Mrs. Parker, how have you been?
#Person2#: Hello Dr. Peters. Just fine thank you. Ricky and I are here for his vaccines.
#Person1#: Very well. Let's see, according to his vaccination record, Ricky has received his Polio, Tetanus and Hepatitis B shots. He is 14 months old, so he is due for Hepatitis A, Chickenpox and Measles shots.
#Person2#: What about Rubella and Mumps?
#Person1#: Well, I can only give him these for now, and after a couple of weeks I can administer the rest.
#Person2#: OK, great. Doctor, I think I also may need a Tetanus booster. Last time I got it was maybe fifteen years ago!
#Person1#: We will check our records and I'll have the nurse administer and the booster as well. Now, please hold Ricky's arm tight, this may sting a little.
---------------------------------------- HIGHLIGHTS ----------------------------------------

Mrs Parker takes Ricky for his vaccines

In [10]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

### Evaluate

In [11]:
import evaluate
rouge = evaluate.load("rouge")

Then create a function that passes your predictions and labels to [compute](https://huggingface.co/docs/evaluate/main/en/package_reference/main_classes#evaluate.EvaluationModule.compute) to calculate the ROUGE metric:

In [12]:
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

### Train

In [13]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/Models/CNN_Model",
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,1.667530,0.328800,0.096900,0.274100,0.274500,18.793000
2,No log,1.609085,0.341000,0.105000,0.282700,0.283000,18.769000
3,No log,1.611374,0.342000,0.103500,0.282900,0.283400,18.841000
4,No log,1.605468,0.340600,0.104300,0.281000,0.281200,18.763000


TrainOutput(global_step=252, training_loss=1.5461866590711806, metrics={'train_runtime': 178.1463, 'train_samples_per_second': 22.453, 'train_steps_per_second': 1.415, 'total_flos': 450853146722304.0, 'train_loss': 1.5461866590711806, 'epoch': 4.0})

In [16]:
trainer.save_model()

In [17]:
type(model)

transformers.models.t5.modeling_t5.T5ForConditionalGeneration

### 3. Evaluate the abstractive and extractive summarization on 2-3 examples and comment on your observations

###Example 1

In [49]:
text =  test_dataset[6]['dialogue']
summary = test_dataset[6]["summary"]

Article 

In [50]:
text

"#Person1#: Kate, you never believe what's happened.\n#Person2#: What do you mean?\n#Person1#: Masha and Hero are getting divorced.\n#Person2#: You are kidding. What happened?\n#Person1#: Well, I don't really know, but I heard that they are having a separation for 2 months, and filed for divorce.\n#Person2#: That's really surprising. I always thought they are well matched. What about the kids? Who get custody?\n#Person1#: Masha, it seems quiet and makable, no quarrelling about who get the house and stock and then contesting the divorce with other details worked out.\n#Person2#: That's the change from all the back stepping we usually hear about. Well, I still can't believe it, Masha and Hero, the perfect couple. When would they divorce be final?\n#Person1#: Early in the New Year I guess."

Actual Summary 


In [51]:
summary

'#Person1# tells Kate that Masha and Hero get divorced. Kate is surprised because she thought they are perfect couple.'

###Example 2

In [56]:
text_eval1 =  test_dataset[12]['dialogue']
summary_eval1 = test_dataset[12]["summary"]
text_eval1

"#Person1#: This Olympic park is so big!\n#Person2#: Yes. Now we are in the Olympic stadium, the center of this park.\n#Person1#: Splendid! When is it gonna be finished?\n#Person2#: The whole stadium is to be finished this June.\n#Person1#: How many seats are there in the stand?\n#Person2#: Oh, there are 5000 seats in total.\n#Person1#: I didn ' t know it would be so big!\n#Person2#: It is! Look there, those are the tracks. And the jumping pit is over there.\n#Person1#: Ah... I see. Hey, look the sign here, No climbing.\n#Person2#: We put many signs with English translations for foreign visitors."

In [57]:
summary_eval1

"#Person1# is surprised at the Olympic Stadium'volume, capacity and interior setting to #Person1#."

In [22]:
from transformers import pipeline

summarizer = pipeline("summarization", model="/content/drive/MyDrive/Models/CNN_Model")
summarizer(text)

[{'summary_text': 'Kate and Masha are getting divorced. They are having a separation for 2 months and filed for divorce. They are quiet and makable, no quarrelling about who get the house and stock and then contest the divorce with other details worked out.'}]

## Observations
(1) While utilizing a more condensed format, the generated summaries are able to convey the key ideas of the discussions.

(2) The summaries accurately rephrase the original dialogues, emphasizing the crucial information communicated and the primary subjects covered.


## Extractive Summarization

Implemeting Q-2 and Q-3 for Extractive Summarization as well

### Example 1 let take the first example ie the 6th dialogue again here

Import Libraries

In [23]:
import numpy as np
import networkx as nx
import sys
from nltk.tokenize.punkt import PunktSentenceTokenizer
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

In [24]:
import matplotlib.pyplot as plt
%matplotlib inline 

In [25]:
def tokenize(document):

    doc_tokenizer = PunktSentenceTokenizer()
    sentences_list = doc_tokenizer.tokenize(document)
    return sentences_list

In [26]:
print(text)

#Person1#: Kate, you never believe what's happened.
#Person2#: What do you mean?
#Person1#: Masha and Hero are getting divorced.
#Person2#: You are kidding. What happened?
#Person1#: Well, I don't really know, but I heard that they are having a separation for 2 months, and filed for divorce.
#Person2#: That's really surprising. I always thought they are well matched. What about the kids? Who get custody?
#Person1#: Masha, it seems quiet and makable, no quarrelling about who get the house and stock and then contesting the divorce with other details worked out.
#Person2#: That's the change from all the back stepping we usually hear about. Well, I still can't believe it, Masha and Hero, the perfect couple. When would they divorce be final?
#Person1#: Early in the New Year I guess.


### Generate a list of sentences in the document


In [27]:
sentences_list = tokenize(text)

for i in sentences_list:
    print(i)

#Person1#: Kate, you never believe what's happened.
#Person2#: What do you mean?
#Person1#: Masha and Hero are getting divorced.
#Person2#: You are kidding.
What happened?
#Person1#: Well, I don't really know, but I heard that they are having a separation for 2 months, and filed for divorce.
#Person2#: That's really surprising.
I always thought they are well matched.
What about the kids?
Who get custody?
#Person1#: Masha, it seems quiet and makable, no quarrelling about who get the house and stock and then contesting the divorce with other details worked out.
#Person2#: That's the change from all the back stepping we usually hear about.
Well, I still can't believe it, Masha and Hero, the perfect couple.
When would they divorce be final?
#Person1#: Early in the New Year I guess.


### Generate term-document matrix (TD matrix) of the data 

In [28]:
cv = CountVectorizer()
cv_matrix = cv.fit_transform(sentences_list)

In [29]:
print('The data type of bow matrix {}'.format(type(cv_matrix)))
print('Shape of the matrix {}'.format(cv_matrix.get_shape))
print('Size of the matrix is: {}'.format(sys.getsizeof(cv_matrix)))
print(cv.get_feature_names_out())
print(cv_matrix.toarray())

The data type of bow matrix <class 'scipy.sparse._csr.csr_matrix'>
Shape of the matrix <bound method spmatrix.get_shape of <15x77 sparse matrix of type '<class 'numpy.int64'>'
	with 118 stored elements in Compressed Sparse Row format>>
Size of the matrix is: 48
['about' 'all' 'always' 'and' 'are' 'back' 'be' 'believe' 'but' 'can'
 'change' 'contesting' 'couple' 'custody' 'details' 'divorce' 'divorced'
 'do' 'don' 'early' 'filed' 'final' 'for' 'from' 'get' 'getting' 'guess'
 'happened' 'having' 'hear' 'heard' 'hero' 'house' 'in' 'it' 'kate'
 'kidding' 'kids' 'know' 'makable' 'masha' 'matched' 'mean' 'months'
 'never' 'new' 'no' 'other' 'out' 'perfect' 'person1' 'person2'
 'quarrelling' 'quiet' 'really' 'seems' 'separation' 'stepping' 'still'
 'stock' 'surprising' 'that' 'the' 'then' 'they' 'thought' 'usually' 'we'
 'well' 'what' 'when' 'who' 'with' 'worked' 'would' 'year' 'you']
[[0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 .

In [30]:
normal_matrix = TfidfTransformer().fit_transform(cv_matrix)
print(normal_matrix.toarray())

[[0.         0.         0.         ... 0.         0.         0.34600868]
 [0.         0.         0.         ... 0.         0.         0.40913323]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.43848789 0.         0.        ]
 [0.         0.         0.         ... 0.         0.41424569 0.        ]]


In [31]:
print(normal_matrix.T.toarray)
res_graph = normal_matrix * normal_matrix.T

<bound method _cs_matrix.toarray of <77x15 sparse matrix of type '<class 'numpy.float64'>'
	with 118 stored elements in Compressed Sparse Column format>>


In [32]:
nx_graph = nx.from_scipy_sparse_array(res_graph)
print('Number of edges {}'.format(nx_graph.number_of_edges()))
print('Number of vertices {}'.format(nx_graph.number_of_nodes()))

Number of edges 62
Number of vertices 15


### Getting the rank of every sentence using textrank

In [33]:
ranks = nx.pagerank(nx_graph)
print(type(ranks))
print('The size used by the dictionary in Bytes is: {}'.format(sys.getsizeof(ranks)))
for i in ranks:
    print(i, ranks[i])


<class 'dict'>
The size used by the dictionary in Bytes is: 640
0 0.0763639752690754
1 0.07173361327726077
2 0.06989284461698039
3 0.07068790281311965
4 0.06470245156929788
5 0.07230766648681372
6 0.06013957395714747
7 0.06084214922407663
8 0.07207824901772494
9 0.054216999521922
10 0.07871100961136426
11 0.0669536871051993
12 0.06960852896166464
13 0.054915931295653496
14 0.056845417272699865


### Finding important sentences and generating summary

In [34]:
sentence_array = sorted(((ranks[i], s) for i, s in enumerate(sentences_list)), reverse=True)
sentence_array = np.asarray(sentence_array)

In [35]:
rank_max = float(sentence_array[0][0])
rank_min = float(sentence_array[len(sentence_array) - 1][0])

In [36]:
print(rank_max)
print(rank_min)

0.07871100961136426
0.054216999521922


In [37]:
temp_array = []

flag = 0
if rank_max - rank_min == 0:
    temp_array.append(0)
    flag = 1

if flag != 1:
    for i in range(0, len(sentence_array)):
        temp_array.append((float(sentence_array[i][0]) - rank_min) / (rank_max - rank_min))

print(len(temp_array))


15


In [38]:
threshold = (sum(temp_array) / len(temp_array)) + 0.2

In [39]:
sentence_list = []
if len(temp_array) > 1:
    for i in range(0, len(temp_array)):
        if temp_array[i] > threshold:
                sentence_list.append(sentence_array[i][1])
else:
    sentence_list.append(sentence_array[0][1])

In [40]:
model = sentence_list

In [41]:
summary = " ".join(str(x) for x in sentence_list)
print("-"*100)
for sentences in summary.split("."):
  print(sentences)

----------------------------------------------------------------------------------------------------
#Person1#: Masha, it seems quiet and makable, no quarrelling about who get the house and stock and then contesting the divorce with other details worked out
 #Person1#: Kate, you never believe what's happened
 #Person1#: Well, I don't really know, but I heard that they are having a separation for 2 months, and filed for divorce
 What about the kids? #Person2#: What do you mean?


###Example 2: Lets check for 300th dialogue

In [59]:
text2 =  test_dataset[300]['dialogue']
summary2 = test_dataset[300]["summary"]

In [60]:
print(text2)

#Person1#: I cannot imagine if Trump were to be our President again.
#Person2#: I am proud to say that he is our President, and I will be really happy if he could be re-elected.
#Person1#: You voted for him, right?
#Person2#: Did you vote for him, because I know that I did.
#Person1#: I am not sure about this.
#Person2#: I have nothing but faith in Trump.
#Person1#: What?
#Person2#: I am pretty sure he will make America great again!
#Person1#: Well, though we do need some change in this country, I don't think he is the right person.
#Person2#: Our country is already changing as it is.
#Person1#: You are right about this.
#Person2#: I trust that he will take good care of our country.
#Person1#: Well, I don't think so. I will vote for Biden anyway.


In [61]:
sentences_list = tokenize(text2)
for i in sentences_list:
    print(i)


#Person1#: I cannot imagine if Trump were to be our President again.
#Person2#: I am proud to say that he is our President, and I will be really happy if he could be re-elected.
#Person1#: You voted for him, right?
#Person2#: Did you vote for him, because I know that I did.
#Person1#: I am not sure about this.
#Person2#: I have nothing but faith in Trump.
#Person1#: What?
#Person2#: I am pretty sure he will make America great again!
#Person1#: Well, though we do need some change in this country, I don't think he is the right person.
#Person2#: Our country is already changing as it is.
#Person1#: You are right about this.
#Person2#: I trust that he will take good care of our country.
#Person1#: Well, I don't think so.
I will vote for Biden anyway.


In [62]:
cv = CountVectorizer()
cv_matrix = cv.fit_transform(sentences_list)

In [63]:

print('The data type of bow matrix {}'.format(type(cv_matrix)))
print('Shape of the matrix {}'.format(cv_matrix.get_shape))
print('Size of the matrix is: {}'.format(sys.getsizeof(cv_matrix)))
print(cv.get_feature_names_out())
print(cv_matrix.toarray())

The data type of bow matrix <class 'scipy.sparse._csr.csr_matrix'>
Shape of the matrix <bound method spmatrix.get_shape of <14x73 sparse matrix of type '<class 'numpy.int64'>'
	with 123 stored elements in Compressed Sparse Row format>>
Size of the matrix is: 48
['about' 'again' 'already' 'am' 'america' 'and' 'anyway' 'are' 'as' 'be'
 'because' 'biden' 'but' 'cannot' 'care' 'change' 'changing' 'could'
 'country' 'did' 'do' 'don' 'elected' 'faith' 'for' 'good' 'great' 'happy'
 'have' 'he' 'him' 'if' 'imagine' 'in' 'is' 'it' 'know' 'make' 'need'
 'not' 'nothing' 'of' 'our' 'person' 'person1' 'person2' 'president'
 'pretty' 'proud' 're' 'really' 'right' 'say' 'so' 'some' 'sure' 'take'
 'that' 'the' 'think' 'this' 'though' 'to' 'trump' 'trust' 'vote' 'voted'
 'we' 'well' 'were' 'what' 'will' 'you']
[[0 1 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]]


In [64]:
normal_matrix = TfidfTransformer().fit_transform(cv_matrix)
print(normal_matrix.toarray())

[[0.         0.3009501  0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.16284314 0.        ]
 [0.         0.         0.         ... 0.         0.         0.39401162]
 ...
 [0.         0.         0.         ... 0.         0.24638973 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.35583415 0.        ]]


In [65]:
print(normal_matrix.T.toarray)
res_graph = normal_matrix * normal_matrix.T

<bound method _cs_matrix.toarray of <73x14 sparse matrix of type '<class 'numpy.float64'>'
	with 123 stored elements in Compressed Sparse Column format>>


In [66]:
nx_graph = nx.from_scipy_sparse_array(res_graph)
print('Number of edges {}'.format(nx_graph.number_of_edges()))
print('Number of vertices {}'.format(nx_graph.number_of_nodes()))

Number of edges 69
Number of vertices 14


In [67]:
ranks = nx.pagerank(nx_graph)
print(type(ranks))
print('The size used by the dictionary in Bytes is: {}'.format(sys.getsizeof(ranks)))
for i in ranks:
    print(i, ranks[i])


<class 'dict'>
The size used by the dictionary in Bytes is: 640
0 0.07206335411974857
1 0.0803877478660073
2 0.08019298874427624
3 0.07056333519987447
4 0.07561903090966403
5 0.058739701175290086
6 0.06284303442092629
7 0.07184042857238535
8 0.08297317127405233
9 0.06458354002198283
10 0.0804776744042729
11 0.07244446081143804
12 0.06525125194704794
13 0.06202028053303349


In [68]:
sentence_array = sorted(((ranks[i], s) for i, s in enumerate(sentences_list)), reverse=True)
sentence_array = np.asarray(sentence_array)

In [69]:
rank_max = float(sentence_array[0][0])
rank_min = float(sentence_array[len(sentence_array) - 1][0])

In [70]:
print(rank_max)
print(rank_min)

0.08297317127405233
0.058739701175290086


In [71]:
temp_array = []

# if all sentences have equal ranks, means they are all the same
# taking any sentence will give the summary, say the first sentence
flag = 0
if rank_max - rank_min == 0:
    temp_array.append(0)
    flag = 1

# If the sentence has different ranks
if flag != 1:
    for i in range(0, len(sentence_array)):
        temp_array.append((float(sentence_array[i][0]) - rank_min) / (rank_max - rank_min))

print(len(temp_array))

14


In [72]:
sentence_array

array([['0.08297317127405233',
        "#Person1#: Well, though we do need some change in this country, I don't think he is the right person."],
       ['0.0804776744042729', '#Person1#: You are right about this.'],
       ['0.0803877478660073',
        '#Person2#: I am proud to say that he is our President, and I will be really happy if he could be re-elected.'],
       ['0.08019298874427624', '#Person1#: You voted for him, right?'],
       ['0.07561903090966403', '#Person1#: I am not sure about this.'],
       ['0.07244446081143804',
        '#Person2#: I trust that he will take good care of our country.'],
       ['0.07206335411974857',
        '#Person1#: I cannot imagine if Trump were to be our President again.'],
       ['0.07184042857238535',
        '#Person2#: I am pretty sure he will make America great again!'],
       ['0.07056333519987447',
        '#Person2#: Did you vote for him, because I know that I did.'],
       ['0.06525125194704794', "#Person1#: Well, I don't think 

In [73]:
threshold = (sum(temp_array) / len(temp_array)) + 0.2
threshold

0.7236092974538315

In [74]:
sentence_list = []
if len(temp_array) > 1:
    for i in range(0, len(temp_array)):
        if temp_array[i] > threshold:
                sentence_list.append(sentence_array[i][1])
else:
    sentence_list.append(sentence_array[0][1])

In [75]:
model = sentence_list

In [76]:
summary = " ".join(str(x) for x in sentence_list)
print("-"*100)
for sentences in summary.split("."):
  print(sentences)

----------------------------------------------------------------------------------------------------
#Person1#: Well, though we do need some change in this country, I don't think he is the right person
 #Person1#: You are right about this
 #Person2#: I am proud to say that he is our President, and I will be really happy if he could be re-elected
 #Person1#: You voted for him, right?
